In [ ]:
'''
1. Доступны только российские тикеры
'''

In [1]:
import pandas as pd
from datetime import timedelta
import sqlite3
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from dateutil.relativedelta import relativedelta

***Функции***

In [2]:
# Цены

In [3]:
def validate(date_text):
    try:
        datetime.strptime(date_text, '%Y-%m-%d')
        return True
    except ValueError:
        print('Введен неправильный формат даты, должен быть 1900-01-01')
        return False

In [4]:
def check_tickers():
    conn = sqlite3.connect('Rumblerg_database.db')
    cursor = conn.cursor()
    
    cursor.execute('''
        SELECT
            TICKER,
            MAX(DATE) as DATE
        FROM price_hist
        GROUP BY TICKER
    ''')

    dict_tickers = []

    for row in cursor.fetchall():
        dict_tickers.append(row[0])
        
    return(dict_tickers)

In [5]:
def get_prices():
    
    act = input('Введите команду (INFO - доступные тикеры, ' +\
    'ALL - все тикеры, DEF - определенные тикеры): ')
    act = act.upper()
    if act == 'INFO':
        print('Доступные тикеры: ' + str(check_tickers()))
        return act
    else:
        if act == 'ALL':
            ticker = check_tickers()
            ticker = set([tick.upper() for tick in ticker])
        if act == 'DEF':
            ticker = input('Введите тикеры через запятую: ').upper().split(',')
            if len(ticker) == 1:
                ticker.append(ticker[0])
        ticker = tuple(ticker)
        start_date = input('Введите дату начала в формате 1900-01-01 ')
        end_date = input('Введите дату окончания в формате 1900-01-01 ')
        if validate(start_date) and validate(end_date):  
            conn = sqlite3.connect('Rumblerg_database.db')
            cursor = conn.cursor()

            dataf = pd.read_sql_query(f'''
                SELECT
                    *
                FROM price_hist
                WHERE upper(TICKER) in {ticker}
                AND DATE >= datetime('{start_date}')
                AND DATE <= datetime('{end_date}')
            ''', conn)

            dataf['DATE'] = pd.to_datetime(dataf['DATE'])
            dataf = dataf.sort_values(by=['TICKER', 'DATE'], ascending=True)
            return dataf
        else:
            return act

In [6]:
def normalise(dataf):
    for comp in dataf['TICKER'].unique():
        dataf.loc[dataf['TICKER'] == comp, 'CLOSE'] /= dataf.loc[dataf['TICKER'] == comp, 'CLOSE'].iloc[0]

In [7]:
def price_chart():
    norm = input('Нормированный график? Y/N')
    norm = norm.upper()
    if norm == 'Y':
        normalise(dataf)
    sns.lineplot(x=dataf['DATE'], y=dataf['CLOSE'], hue=dataf['TICKER'])
    plt.title('Динамика цен')
    plt.legend(loc='upper center', bbox_to_anchor=(1.15, 1.02))
    plt.xlabel("")
    plt.ylabel("")
    plt.savefig('price_chart.jpg', bbox_inches='tight')
    plt.show()

In [8]:
def prices_to_returns(dataf):
    dataf_ret = dataf.copy()
    
    for comp in dataf_ret['TICKER'].unique():
        dataf_ret.loc[dataf_ret['TICKER'] == comp, 'CLOSE'] /= dataf_ret.loc[dataf_ret['TICKER'] == comp, 'CLOSE'].shift(1)
        dataf_ret.loc[dataf_ret['TICKER'] == comp, 'CLOSE'] = np.log(dataf_ret.loc[dataf_ret['TICKER'] == comp, 'CLOSE'])
    dataf_ret = dataf_ret.dropna()

    returns = pd.DataFrame()
    first_ticker = dataf_ret['TICKER'].unique()[0]
    returns[['DATE', first_ticker]] = dataf_ret[dataf_ret['TICKER'] == first_ticker][['DATE', 'CLOSE']]
    for comp in dataf_ret['TICKER'].unique()[1:]:
        returns_new = pd.DataFrame()
        returns_new[['DATE', comp]] = dataf_ret[dataf_ret['TICKER'] == comp][['DATE', 'CLOSE']]
        returns = returns.merge(returns_new, how='left')
    returns = returns.fillna(0)
    
    return returns

In [9]:
def corr_matrix(returns):
    returns.iloc[:, 1:].corr().to_excel('corr_matrix.xlsx')
    sns.heatmap(returns.iloc[:, 1:].corr(), annot = True)
    plt.savefig('corr_matrix.jpg')
    plt.show()

In [10]:
def returns_vol(returns):
    stats = pd.DataFrame()
    stats['Дох-ть за период %'] = np.exp(returns.iloc[:, 1:].sum()) - 1
    term = ((returns['DATE'].iloc[-1] - returns['DATE'].iloc[0]).days / 365)
    stats['Дох-ть за период %гг'] = np.exp(returns.iloc[:, 1:].sum() / term) - 1
    stats['Средняя вол-ть %гг'] = returns.iloc[:, 1:].std()*(250**(1/2))
    stats.to_excel('returns_vol.xlsx')
    
    for col in stats.columns:
        stats[col] = pd.Series(["{0:.2f}%".format(val * 100) for val in stats[col]], index = stats.index)
        
    fig, ax = plt.subplots()
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=stats.values, colLabels=stats.columns, rowLabels=stats.index, loc="center")
    table.scale(1.5,1.5)
    table.auto_set_font_size(False)
    table.set_fontsize(12)

    cells = table.properties()["celld"]
    for i in range(len(stats) + 1):
        for j in range(len(stats.columns)):
            cells[i, j].set_text_props(ha="center")

    plt.savefig('returns_vol.jpg', bbox_inches='tight')
    plt.show()

In [11]:
def returns_table(returns, dataf):
    end_date = returns['DATE'].iloc[-1]
    start_date_w = end_date + relativedelta(weeks=-1)
    start_date_m = end_date + relativedelta(months=-1)
    start_date_y = end_date + relativedelta(years=-1)
    comps = returns.columns[1:].to_list()
    ret_stat = pd.DataFrame()
    rets1w = np.exp(returns[returns['DATE'] > start_date_w].sum())-1
    rets1m = np.exp(returns[returns['DATE'] > start_date_m].sum())-1
    rets1y = np.exp(returns[returns['DATE'] > start_date_y].sum())-1

    ret_stat['1 н'] = rets1w
    ret_stat['1 м'] = rets1m
    ret_stat['YoY'] = rets1y

    current_price = dataf[dataf['DATE'] == end_date][['TICKER', 'CLOSE']]
    current_price.index = current_price['TICKER']
    current_price = current_price['CLOSE']
    ret_stat['CLOSE'] = current_price
    ret_stat = ret_stat[['CLOSE', '1 н', '1 м', 'YoY']]

    ret_stat.to_excel('returns_table.xlsx')

    for col in ret_stat.columns[1:]:
        ret_stat[col] = pd.Series(["{0:.2f}%".format(val * 100) for val in ret_stat[col]], index = ret_stat.index)

    fig, ax = plt.subplots()
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=ret_stat.values, colLabels=ret_stat.columns, rowLabels=ret_stat.index, loc="center")
    table.scale(1.5,1.5)
    table.auto_set_font_size(False)
    table.set_fontsize(12)

    cells = table.properties()["celld"]
    for i in range(len(ret_stat) + 1):
        for j in range(len(ret_stat.columns)):
            cells[i, j].set_text_props(ha="center")

    plt.savefig('returns_table.jpg', bbox_inches='tight')
    plt.show()

In [12]:
# Отчетности

In [13]:
def check_tickers_fin():
    conn = sqlite3.connect('Rumblerg_database.db')
    cursor = conn.cursor()
    
    cursor.execute('''
        SELECT
            Тикер,
            MAX(Дата) as Дата
        FROM MSFO_data_calc
        GROUP BY Тикер
    ''')

    dict_tickers = []

    for row in cursor.fetchall():
        dict_tickers.append(row[0])
        
    return(dict_tickers)

In [14]:
def check_fins_cols():
    data = pd.read_sql_query(f'''
        SELECT
            *
        FROM MSFO_data_calc
    ''', conn)
    return data.columns.to_list()[4:]

In [15]:
def get_financials():
    
    act = input('Введите команду (INFO - доступные тикеры, ' +\
    'ALL - все тикеры, DEF - определенные тикеры): ')
    act = act.upper()
    if act == 'INFO':
        print('Доступные тикеры: ' + str(check_tickers_fin()))
        return act
    else:
        if act == 'ALL':
            ticker = check_tickers_fin()
        if act == 'DEF':
            ticker = input('Введите тикеры через запятую: ').upper().split(',')
            if len(ticker) == 1:
                ticker.append(ticker[0])
        ticker = tuple(ticker)
        start_date = input('Введите дату начала в формате 1900-01-01' )
        end_date = input('Введите дату окончания в формате 1900-01-01' )
        rolling = input('Выводить скользящие промежуточные показатели за год (Y/N)? ')
        rolling = rolling.upper()
        if validate(start_date) and validate(end_date):  
            conn = sqlite3.connect('Rumblerg_database.db')
            cursor = conn.cursor()
            
            if rolling == 'Y':
                table = 'MSFO_data_calc'
            else:
                table = 'MSFO_data'
            
            dataf = pd.read_sql_query(f'''
                SELECT
                    *
                FROM {table}
                WHERE Тикер in {ticker}
                AND Дата >= '{start_date}'
                AND Дата <= '{end_date}'
            ''', conn)

            dataf['Дата'] = pd.to_datetime(dataf['Дата'])
            dataf = dataf.sort_values(by=['Тикер', 'Дата'], ascending=True)
            return dataf

In [16]:
def fins_chart(dataf):
    params = input('Введите необходимые параметры через запятую (ALL - все): ' +\
    '(доступны: ' + str(check_fins_cols()).replace("'", "") + ') ').split(',')
    if params[0].upper() == 'ALL':
        params = check_fins_cols()
    ticker = list(dataf['Тикер'].unique())
    frequency = input('Использовать только годовые данные? (Y/N)').upper()
    if frequency == 'Y':
        dataf = dataf[[str(date).__contains__('-12-') for date in dataf['Дата']]]
    for tick in ticker:
        for param in params:
            temp_dataf = dataf[dataf['Тикер'] == tick][['Дата', param]].sort_values(by=['Дата'])
            
            digits = len(str(int(temp_dataf[param].max())))
            if digits < 4:
                temp_dataf[param] = temp_dataf[param]
                dig = 'руб'
            if 4 <= digits < 7:
                temp_dataf[param] = temp_dataf[param] / 1000
                dig = 'тыс. руб'
            if 7 <= digits < 10:
                temp_dataf[param] = temp_dataf[param] / 1000000
                dig = 'млн. руб'
            if 10 <= digits < 13:
                temp_dataf[param] = temp_dataf[param] / 1000000000
                dig = 'млрд. руб'
            if 13 <= digits < 17:
                temp_dataf[param] = temp_dataf[param] / 1000000000000
                dig = 'трлн. руб'
                
            if len(temp_dataf) > 30:
                rot = 90
            elif len(temp_dataf) > 15:
                rot = 45
            else:
                rot = 30
            ax = sns.barplot(x=[date.strftime('%Y-%m') for date in temp_dataf['Дата']], y=temp_dataf[param], color='limegreen') #purple
            ax.set_xticklabels(ax.get_xticklabels(), rotation = rot)
            for container in ax.containers:
                ax.bar_label(container, fmt='%.0f', rotation=90, label_type='center', color='black') #gold #springgreen #lime #aqua
            plt.title(param + ", " + tick)
            plt.xlabel("")
            plt.ylabel(dig)
            plt.savefig(param + '_' + tick + 'jpg', bbox_inches='tight')
            plt.show()

In [17]:
def check_tickers_divs():
    conn = sqlite3.connect('Rumblerg_database.db')
    cursor = conn.cursor()
    
    cursor.execute('''
        SELECT
            TICKER
        FROM div_comps
    ''')

    div_tickers = []

    for row in cursor.fetchall():
        div_tickers.append(row[0])
        
    return(div_tickers)

In [18]:
def df_divs():
    act = input('Введите команду (INFO - доступные тикеры, ' +\
    'ALL - все тикеры, DEF - определенные тикеры): ')
    act = act.upper()
    div_tickers = check_tickers_divs()
    if act == 'ALL':
        tickers = div_tickers
    if act == 'INFO':
        print('Доступные тикеры: ' + str(div_tickers))
        return act
    if act == 'DEF':
        tickers = input('Введите тикеры через запятую: ').split(',')
        tickers = [ticker.upper() for ticker in tickers if ticker.upper() in div_tickers]        
    if len(tickers) == 0:
        print('Тикеров среди доступных нет')
        return act
    else:
        conn = sqlite3.connect('Rumblerg_database.db')
        
        data = pd.read_sql_query(f'''
            SELECT
                *
            FROM div_hist
            WHERE TICKER in {tuple(tickers)}
        ''', conn)
        
        data['DATE'] = [datetime.strptime(day[:10], '%Y-%m-%d') for day in data['DATE']]
        
        return data

In [19]:
# Функции для коэф-тов

In [20]:
def coef_prices_func(ticker, start_date, end_date, path = 'Rumblerg_database.db'):
    conn = sqlite3.connect(path)
    cursor = conn.cursor()

    prices = pd.read_sql_query(f'''
        SELECT
            TICKER,
            DATE,
            CLOSE
        FROM price_hist
        WHERE DATE >= datetime('{start_date}')
        AND DATE <= datetime('{end_date}')
        AND TICKER = '{ticker}'
    ''', conn, parse_dates='DATE')
    
    prices = prices.sort_values(by='DATE', ascending=True)
    
    return prices

In [21]:
def coef_nshares_func(ticker, start_date, end_date, coef_df, path = 'Rumblerg_database.db'):
    conn = sqlite3.connect(path)
    cursor = conn.cursor()

    nshares = pd.read_sql_query(f'''
        SELECT
            DATE,
            N_SHARES
        FROM n_shares
        WHERE DATE >= COALESCE((
            SELECT
                max(DATE)
            FROM n_shares
            WHERE DATE <= datetime('{start_date}')
            AND TICKER = '{ticker}'
        ), datetime('{start_date}'))
        AND DATE <= datetime('{end_date}')
        AND TICKER = '{ticker}'
    ''', conn, parse_dates='DATE', index_col='DATE')
    
    nshares = nshares.sort_values(by='DATE', ascending=True)
    
    coef_df['N_SHARES'] = np.nan
    for ndate in range(len(nshares)):
        coef_df.loc[coef_df['DATE'] >= nshares.index[ndate], 'N_SHARES'] = nshares.iloc[ndate, 0]
        
    return coef_df

In [22]:
def coef_cap_func(coef_df):
    coef_df['CAPITALIZATION'] = coef_df['CLOSE'] * coef_df['N_SHARES']
    
    return coef_df

In [23]:
def fins_for_coefs_func(coef_df, ticker, start_date, end_date, path = 'Rumblerg_database.db'):
    
    conn = sqlite3.connect(path)
    cursor = conn.cursor()
    
    fins_frame = pd.read_sql_query(f'''
        SELECT
            *
        FROM MSFO_data_calc
        WHERE Тикер = '{ticker}'
        AND Дата >= COALESCE((
            SELECT
                max(Дата)
            FROM MSFO_data_calc
            WHERE Тикер = '{ticker}'
            AND Дата <= datetime('{start_date}')
        ), datetime('{start_date}'))
        AND Дата <= datetime('{end_date}')
    ''', conn, parse_dates='Дата')
    
    fins_frame = fins_frame.sort_values(by='Дата', ascending=True)
    
    coef_df['STATEMENT_DATE'] = np.nan
    for ndate in range(len(fins_frame)):
        coef_df.loc[coef_df['DATE'] >= fins_frame.iloc[ndate, 1], 'STATEMENT_DATE'] = fins_frame.iloc[ndate, 1]
    
    return fins_frame, coef_df

In [24]:
def coef_financials_func(fins_for_coefs, coef_df):
    financials_for_coef_list = ['Чистый_долг', 'Выручка', 'EBITDA', 'Чистая_прибыль', 'Операционный_денежный_поток',\
           'Инвестиционный_денежный_поток', 'Финансовый_денежный_поток', 'Капитальные_затраты',\
           'Активы', 'Капитал']
    coef_df[financials_for_coef_list] = np.nan
    
    for fin in financials_for_coef_list:
        for date, value in zip(fins_for_coefs['Дата'], fins_for_coefs[fin]):
            coef_df.loc[coef_df['STATEMENT_DATE'] == date, fin] = value

    return coef_df

In [25]:
def coefs_func(coef_df):
    coef_df['EV'] = coef_df['CAPITALIZATION'] + coef_df['Чистый_долг']
    coef_df['EPS'] = np.where(coef_df['N_SHARES'] != 0, coef_df['Чистая_прибыль'] / coef_df['N_SHARES'],\
                             np.nan)
    coef_df['FCF'] = coef_df['Операционный_денежный_поток'] + coef_df['Инвестиционный_денежный_поток'] +\
    coef_df['Финансовый_денежный_поток'] - coef_df['Капитальные_затраты']
    coef_df['FCF/акцию'] = np.where(coef_df['N_SHARES'] != 0, coef_df['FCF'] / coef_df['N_SHARES'],\
                                    np.nan)
    coef_df['CAPEX/Выручка'] = np.where(coef_df['Выручка'] != 0, coef_df['Капитальные_затраты'] / coef_df['Выручка'],\
                                        np.nan)
    coef_df['Рентабельность_EBITDA'] = np.where(coef_df['Выручка'] != 0, coef_df['EBITDA'] / coef_df['Выручка'],\
                                               np.nan)
    coef_df['Чистая_рентабельность'] = np.where(coef_df['Выручка'] != 0, coef_df['Чистая_прибыль'] / coef_df['Выручка'],\
                                                np.nan)
    coef_df['Доходность_FCF'] = np.where(coef_df['CAPITALIZATION'] != 0, coef_df['FCF'] / coef_df['CAPITALIZATION'],\
                                         np.nan)
    coef_df['EV/EBITDA'] = np.where(coef_df['EBITDA'] != 0, coef_df['EV'] / coef_df['EBITDA'],\
                                    np.nan)
    coef_df['Чистый_долг/EBITDA'] = np.where(coef_df['EBITDA'] != 0, coef_df['Чистый_долг'] / coef_df['EBITDA'],\
                                             np.nan)
    coef_df['ROE'] = np.where(coef_df['Капитал'] != 0, coef_df['Чистая_прибыль'] / coef_df['Капитал'],\
                              np.nan)
    coef_df['ROA'] = np.where(coef_df['Активы'] != 0, coef_df['Чистая_прибыль'] / coef_df['Активы'],\
                              np.nan)
    coef_df['P/E'] = np.where(coef_df['Чистая_прибыль'] != 0, coef_df['CAPITALIZATION'] / coef_df['Чистая_прибыль'],\
                              np.nan)
    coef_df['P/S'] = np.where(coef_df['Выручка'] != 0, coef_df['CAPITALIZATION'] / coef_df['Выручка'],\
                              np.nan)
    coef_df['P/BV'] = np.where(coef_df['Капитал'] != 0, coef_df['CAPITALIZATION'] / coef_df['Капитал'],\
                                np.nan)
    
    return coef_df

In [26]:
def yearly_divs(coef_df, divs_frame, ticker, start_date, end_date, path = 'Rumblerg_database.db'):
    last_div_date_list = []
    last_div_list = []
    div_last_year_list = []
    div_return_last_year_list = []

    start_date_div = str(int(start_date[:4]) - 1) + start_date[4:]
    prices_for_divs = coef_prices_func(ticker, start_date_div, end_date, path = 'Rumblerg_database.db')
    
    for date in coef_df['DATE']:
        if divs_frame[divs_frame['DATE'] <= date].empty == False:
            last_div_date = divs_frame[divs_frame['DATE'] <= date].sort_values(by='DATE', ascending=False).iloc[0, 1]
            last_div = divs_frame[divs_frame['DATE'] <= date].sort_values(by='DATE', ascending=False).iloc[0, 2]

            divs_last_year_df = divs_frame[(divs_frame['DATE'] <= date) & (divs_frame['DATE'] >= date - relativedelta(years=1))]
            divs_last_year_df['CLOSE'] = np.nan

            nan_values = False
            for nrow in range(len(divs_last_year_df)):
                div_pr = prices_for_divs[prices_for_divs['DATE'] <= divs_last_year_df.iloc[nrow, 1]]
                if div_pr.empty:
                    divs_last_year_df.iloc[nrow, 3] = np.nan
                    nan_values = True
                else:
                    divs_last_year_df.iloc[nrow, 3] = div_pr.iloc[-1, 2]
            divs_last_year_df['DIV_RETURN'] = divs_last_year_df['DIVIDEND'] / divs_last_year_df['CLOSE']

            div_last_year = divs_last_year_df['DIVIDEND'].sum()
            if nan_values:
                div_return_last_year = np.nan
            else:
                div_return_last_year = divs_last_year_df['DIV_RETURN'].sum()

        else:
            last_div_date = np.nan
            last_div = np.nan
            div_last_year = np.nan
            div_return_last_year = np.nan

        last_div_date_list.append(last_div_date)
        last_div_list.append(last_div)
        div_last_year_list.append(div_last_year)
        div_return_last_year_list.append(div_return_last_year)
        
    coef_df['LAST_DIV_DATE'] = last_div_date_list
    coef_df['LAST_DIV'] = last_div_list
    coef_df['LAST_YEAR_DIVS'] = div_last_year_list
    coef_df['LAST_YEAR_DIVS_RETURN'] = div_return_last_year_list
        
    return coef_df

In [27]:
def divs_for_coef(coef_df, ticker, start_date, end_date, path = 'Rumblerg_database.db'):
    conn = sqlite3.connect(path)
    cursor = conn.cursor()
    
    start_date_div = str(int(start_date[:4]) - 1) + start_date[4:]
    change = pd.read_sql_query('''
        SELECT
            *
        FROM div_tickers_change
    ''', conn)
    old_list = list(change.iloc[:, 0])
    new_list = list(change.iloc[:, 1])
    
    if ticker in new_list:
        ticker = old_list[new_list.index(ticker)]
    
    divs_aval = pd.read_sql_query(f'''
        SELECT
            *
        FROM div_comps
        WHERE TICKER = '{ticker}'
    ''', conn)
    
    divs_frame = pd.read_sql_query(f'''
        SELECT
            *
        FROM div_hist
        WHERE TICKER = '{ticker}'
        AND DATE >= datetime('{start_date_div}')
        AND DATE <= datetime('{end_date}')
    ''', conn, parse_dates='DATE')
    
    if ticker in old_list:
        divs_frame['TICKER'] = new_list[old_list.index(ticker)]
        
    divs_aval = divs_aval.empty == False
    
    if divs_aval == False:
        coef_df['LAST_DIV_DATE'] = 'Тикера нет в базе дивидендов'
        coef_df['LAST_DIV'] = np.nan
        coef_df['LAST_YEAR_DIVS'] = np.nan
        coef_df['LAST_YEAR_DIVS_RETURN'] = np.nan
    else:
        coef_df = yearly_divs(coef_df, divs_frame, ticker, start_date_div, end_date)
    
    return coef_df

In [28]:
def fins_coef():
    act = input('Введите команду (INFO - доступные тикеры, ' +\
    'ALL - все тикеры, DEF - определенные тикеры): ')
    act = act.upper()
    tickers_fin = check_tickers_fin()
    tickers_price = check_tickers()
    tickers_aval = [ticker for ticker in tickers_fin if ticker in tickers_price]
    if act == 'INFO':
        print('Доступные тикеры: ' + str(tickers_aval))
        return act
    elif act == 'ALL':
        tickers = tickers_aval
    elif act == 'DEF':
        tickers = input('Введите тикеры через запятую: ').split(',')
        tickers = [ticker.upper() for ticker in tickers if ticker.upper() in tickers_aval]
    if len(tickers) == 0:
        print('Тикеров среди доступных нет')
        return act
    else:
        start_date = input('Введите дату начала в формате 1900-01-01' )
        end_date = input('Введите дату окончания в формате 1900-01-01' )
        if validate(start_date) and validate(end_date):
            df = pd.DataFrame()
            for ticker_f in tickers:
                coef_df = coef_prices_func(ticker_f, start_date, end_date)
                coef_df = coef_nshares_func(ticker_f, start_date, end_date, coef_df)
                coef_df = coef_cap_func(coef_df)
                fins_for_coefs, coef_df = fins_for_coefs_func(coef_df, ticker_f, start_date, end_date)
                coef_df = coef_financials_func(fins_for_coefs, coef_df)
                coef_df = coefs_func(coef_df)
                coef_df = divs_for_coef(coef_df, ticker_f, start_date, end_date)
                df = df.append(coef_df)
            return df

***Функционал***

In [29]:
# Добавить возможность указывать даты выгрузки и инструменты + order by Дата
comm = 'Добро пожаловать!'
print(comm)
while comm != 'EXIT':
    comm = input('Введите команду (EP - выгрузка цен в эксель, CP - график цен, '\
                 + 'SP - статистики по ценам, EF - выгрузка фин. показателей в эксель, '\
                 + 'CF - график фин. показателей, ED - выгрузка дивидендов в эксель, '\
                 + 'CC - расчет и выгрузка фин. коэф-тов, EXIT - выход): ')
    comm = comm.upper()
    if comm == 'EP':
        dataf = get_prices()
        if isinstance(dataf, pd.DataFrame):
            dataf.to_excel('price_hist.xlsx', index=False)
    if comm == 'CP':
        dataf = get_prices()
        if isinstance(dataf, pd.DataFrame):
            price_chart()
    if comm == 'SP':
        comm2 = input('Введите команду (COR - корреляции, DISK - статистики, RET - доходности')
        comm2 = comm2.upper()

        dataf = get_prices()
        if isinstance(dataf, pd.DataFrame):
            returns = prices_to_returns(dataf)
            if comm2 == 'COR':
                corr_matrix(returns)
            if comm2 == 'DISK':
                returns_vol(returns)      
            if ((comm2 == 'RET') | (comm2 == 'RETM')):
                returns_table(returns, dataf)
    if comm == 'EF':
        dataf = get_financials()
        if isinstance(dataf, pd.DataFrame):
            dataf.to_excel('financials_hist.xlsx', index=False)
    if comm == 'CF':
        dataf = get_financials()
        if isinstance(dataf, pd.DataFrame):
            fins_chart(dataf)
    if comm == 'ED':
        dataf = df_divs()
        if isinstance(dataf, pd.DataFrame):
            dataf.to_excel('divs_hist.xlsx', index=False)
    if comm == 'CC':
        dataf = fins_coef()
        if isinstance(dataf, pd.DataFrame):
            dataf.to_excel('coefs_hist.xlsx', index=False)

Добро пожаловать!
Введите команду (EP - выгрузка цен в эксель, CP - график цен, SP - статистики по ценам, EF - выгрузка фин. показателей в эксель, CF - график фин. показателей, ED - выгрузка дивидендов в эксель, CC - расчет и выгрузка фин. коэф-тов, EXIT - выход): ep
Введите команду (INFO - доступные тикеры, ALL - все тикеры, DEF - определенные тикеры): all
Введите дату начала в формате 1900-01-01 2022-01-01
Введите дату окончания в формате 1900-01-01 2022-11-10
Введите команду (EP - выгрузка цен в эксель, CP - график цен, SP - статистики по ценам, EF - выгрузка фин. показателей в эксель, CF - график фин. показателей, ED - выгрузка дивидендов в эксель, CC - расчет и выгрузка фин. коэф-тов, EXIT - выход): exit


In [ ]:
!!!

In [ ]:
AFKS,AFLT,ALRS,CBOM,CHMF,DSKY,ENPG,FIVE,FIXP,GAZP,GLTR,GMKN,HYDR,IRAO,LKOH,MAGN,MGNT,MOEX,MTSS,NLMK,NVTK,OZON,PHOR,PIKK,PLZL,POLY,ROSN,RTKM,RUAL,SBER,SBERP,SNGS,SNGSP,TATN,TATNP,TCSG,TRNFP,VKCO,VTBR,YNDX

In [ ]:
#03-31
#06-30
#09-30
#12-31

In [ ]:
!!!

In [840]:
test = show_table('n_shares')

In [28]:
def show_table(table_name):
    
    conn = sqlite3.connect('Rumblerg_database.db')
    cursor = conn.cursor()
    dataf = pd.read_sql_query(f'''
        SELECT
            *
        FROM '{table_name}'
    ''', conn)

    return dataf

In [ ]:
MSFO_data_calc
price_hist
n_shares
div_comps
div_hist
div_tickers_change